In [1]:
import pickle
import pandas as pd
import glob2
import plotly.express as px
import plotly.graph_objects as go
from influxdb import InfluxDBClient
from plotly.subplots import make_subplots
import log_loader_time
from tqdm import tqdm_notebook
import glob

Welcome to JupyROOT 6.16/00


In [2]:
time={}
for filename, (run,)in glob2.iglob("/media/alb/Removibile/time_save/*", with_matches=True):
        if int(run)>330:
            with open (filename,"rb+") as savefile:
                start,end=pickle.load(savefile)
                time[int(run)]=start


In [3]:
def calculater_middle_time(start_dict,end_dict):
    central_time_dict={}
    for subrun in start_dict.keys():
        central_time_dict[subrun]=start_dict[subrun]+ round((end_dict[subrun]-start_dict[subrun])/2)
    return (central_time_dict)

In [4]:
time={}
for filename, (run,)in glob2.iglob("/media/alb/Removibile/time_save/*", with_matches=True):
        if int(run)>330:
            with open (filename,"rb+") as savefile:
                start,end=pickle.load(savefile)
                calculater_middle_time(start,end)
                time[int(run)]=start


Ricordati di generare i file di tempo (log_loader_time.py)

In [5]:
def extract_from_graal_and_save():
    dict_4_pd={
        "run":[],
        "subrun":[],
        "time":[],
        "Number_of_cluster_L1top_spX":[],
        "Number_of_cluster_L1bot_spX":[],
        "Number_of_cluster_L2top_spX":[],
        "Number_of_cluster_L2bot_spX":[],
        "Number_of_cluster_L1top_spV":[],
        "Number_of_cluster_L1bot_spV":[],
        "Number_of_cluster_L2top_spV":[],
        "Number_of_cluster_L2bot_spV":[],
        "Cluster_size_L1top_spX":[],
        "Cluster_size_L1bot_spX":[],
        "Cluster_size_L2top_spX":[],
        "Cluster_size_L2bot_spX":[],
        "Cluster_size_L1top_spV":[],
        "Cluster_size_L1bot_spV":[],
        "Cluster_size_L2top_spV":[],
        "Cluster_size_L2bot_spV":[],
        "Cluster_charge_L1top_spX":[],
        "Cluster_charge_L1bot_spX":[],
        "Cluster_charge_L2top_spX":[],
        "Cluster_charge_L2bot_spX":[],
        "Cluster_charge_L1top_spV":[],
        "Cluster_charge_L1bot_spV":[],
        "Cluster_charge_L2top_spV":[],
        "Cluster_charge_L2bot_spV":[]
    }
    time={}
    for filename, (run,)in glob2.iglob("/media/alb/Removibile/time_save/*", with_matches=True):
        if int(run)>330:
            with open (filename,"rb+") as savefile:
                start,end=pickle.load(savefile)
            time[int(run)]=calculater_middle_time(start,end)
            tot= len(glob.glob1("/home/alb/GRAAL_out/srv_lab_mount","716392{}_*.txt".format(run)))
            for filename, (subrun,)in tqdm_notebook (glob2.iglob("/home/alb/GRAAL_out/srv_lab_mount/716392{}_*.txt".format(run), with_matches=True), total=tot):
                with open (filename,'r') as graal_file:
                    if subrun in time[int(run)].keys():
                        dict_4_pd["run"].append(int(run))
                        dict_4_pd["subrun"].append(int(subrun))
                        dict_4_pd["time"].append(time[int(run)][subrun])
                        for line in graal_file.readlines():
                            for key in dict_4_pd:
                                if "X" in key:
                                    if key.split("_sp")[0].replace("_"," ") in line:
                                        dict_4_pd["{}_spX".format(key.split("_sp")[0])].append(float(line.split("-->")[1].split("|")[0].strip("\n").strip(" fC ")))
                                        dict_4_pd["{}_spV".format(key.split("_sp")[0])].append(float(line.split("-->")[2].strip("\n").strip(" fC ")))
                    else:
#                         print ("Missing, R{}, S{}".format(run,subrun))
                        pass
            print ("RUN {} done".format(run))
            cluster_pd=pd.DataFrame(dict_4_pd)
            cluster_pd=cluster_pd.sort_values(by=["run","subrun"])
            # cluster_pd['time'] = pd.to_datetime(cluster_pd['time'],unit='s')
            # cluster_pd=cluster_pd.set_index("time")
            # cluster_pd["hour"]=cluster_pd.index.hour
            # cluster_pd["day"]=cluster_pd.index.date
            savefilename="/media/alb/Removibile/graal_extract/cluster_dataframe"
            with open (savefilename,"wb+") as savefile:
                pickle.dump(cluster_pd,savefile)  
    return cluster_pd

In [14]:
cluster_pd=extract_from_graal_and_save()

RUN 337 done


RUN 338 done


RUN 339 done


RUN 341 done


RUN 342 done


...

RUN 411 done


RUN 412 done


RUN 418 done


RUN 419 done


RUN 420 done


In [16]:
cluster_pd['time'] = pd.to_datetime(cluster_pd['time'],unit='s')
cluster_pd=cluster_pd.set_index("time")
cluster_pd["hour"]=cluster_pd.index.hour
cluster_pd["day"]=cluster_pd.index.date

In [17]:
cluster_pd_1h=cluster_pd.resample("1h").mean()

In [18]:
cluster_pd

,run,subrun,Number_of_cluster_L1top_spX,Number_of_cluster_L1bot_spX,Number_of_cluster_L2top_spX,Number_of_cluster_L2bot_spX,Number_of_cluster_L1top_spV,Number_of_cluster_L1bot_spV,Number_of_cluster_L2top_spV,Number_of_cluster_L2bot_spV,...,Cluster_charge_L1top_spX,Cluster_charge_L1bot_spX,Cluster_charge_L2top_spX,Cluster_charge_L2bot_spX,Cluster_charge_L1top_spV,Cluster_charge_L1bot_spV,Cluster_charge_L2top_spV,Cluster_charge_L2bot_spV,hour,day
time,,,,,,,,,,,,,,,,,,,,,
2020-01-05 05:52:43,337,0,83.0,82.0,81.0,79.0,84.0,83.0,87.0,84.0,...,81.8633,99.6370,81.0019,89.6558,55.2624,70.4593,67.8610,58.9870,5,2020-01-05
2020-01-05 06:03:37,337,1,87.0,86.0,83.0,81.0,93.0,90.0,85.0,90.0,...,76.5840,106.6850,77.8015,85.6457,49.6046,76.8946,74.3361,66.1680,6,2020-01-05
2020-01-05 06:14:29,337,2,88.0,91.0,80.0,84.0,95.0,96.0,95.0,93.0,...,85.2310,91.2527,74.1860,88.3277,54.0509,59.2968,70.5279,66.3098,6,2020-01-05
2020-01-05 06:25:20,337,3,73.0,84.0,81.0,80.0,81.0,88.0,82.0,83.0,...,74.4185,103.3830,83.6824,92.8090,54.7576,67.6120,72.0452,74.3060,6,2020-01-05
2020-01-05 06:36:12,337,4,95.0,105.0,93.0,95.0,101.0,104.0,90.0,101.0,...,76.4029,96.1721,80.8067,88.0801,52.2241,73.5097,69.9638,73.4802,6,2020-01-05
2020-01-05 06:47:03,337,5,95.0,100.0,85.0,92.0,100.0,104.0,101.0,104.0,...,84.1363,95.4949,73.6082,75.6886,54.9979,60.1154,65.8073,69.0004,6,2020-01-05
2020-01-05 06:57:56,337,6,96.0,102.0,91.0,94.0,102.0,103.0,92.0,99.0,...,81.5260,102.5170,77.1884,91.6768,51.8572,74.0344,80.7812,76.3782,6,2020-01-05
2020-01-05 07:08:49,337,7,101.0,107.0,100.0,100.0,102.0,107.0,105.0,106.0,...,86.2395,102.8330,77.1871,78.3040,58.2411,70.1369,70.1408,67.4631,7,2020-01-05
2020-01-05 07:19:43,337,8,101.0,106.0,98.0,89.0,107.0,111.0,104.0,101.0,...,77.7380,92.6453,77.3944,86.8064,49.9832,59.7237,74.9835,64.3587,7,2020-01-05
